In [1]:
#import dependencies
import time
import requests
import numpy as np
import pandas as pd
from citipy import citipy
from datetime import datetime
from config import weather_api_key

In [2]:
# generate 2000 random (Latitude, Longitude) coordinates
lats = np.random.uniform(low=-90, high=90, size=2000)
lngs = np.random.uniform(low=-180, high=180, size=2000)
coordinates = list(zip(lats, lngs))

# create a list of cities
cities = []
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    if city not in cities:
        cities.append(city)

In [3]:
# base url for weathermap api calls
base_url = f"https://api.openweathermap.org/data/2.5/weather?units=Imperial&lat=35&lon=139&appid={weather_api_key}"

# initialize city_data
city_data = []

# beginning of data logging
print(f"Beginning Weather Data Retrieval\n{'-'* 32}\n\t\t...\n")

# loop through all the cities
for i, city in enumerate(cities):
    
    # group cities in sets of 50
    if (i % 50 == 0) and (i >= 50):

        time.sleep(60)
        
    # create endpoint URL with each city
    city_url = f"{base_url}&q={city}"
    
    # run an api request for each city
    try:
        # parse the JSON and retrive data
        city_weather = requests.get(city_url).json()

        city_name = city_weather['name']
        city_lat = city_weather['coord']['lat']
        city_lng = city_weather['coord']['lon']
        city_country = city_weather['sys']['country']
        city_max_temp = city_weather['main']['temp_max']
        city_humidity = city_weather['main']['humidity']
        city_cloudiness = city_weather['clouds']['all']
        city_wind_speed = city_weather['wind']['speed']
        city_current_description = city_weather['weather'][0]['description']
        
        # append to city_data list
        city_data.append({
            'City': city_name.title(),
            'Country': city_country,
            'Lat': city_lat, 
            'Lng': city_lng, 
            'Max Temp': city_max_temp,
            'Humidity': city_humidity, 
            'Cloudiness': city_cloudiness, 
            'Wind Speed': city_wind_speed,
            'Current Description': city_current_description})
    except:
        pass

# data logging is complete!
print(f"Weather Data Retrieval is Complete!\n{'-'* 35}")

Beginning Weather Data Retrieval
--------------------------------
		...

Weather Data Retrieval is Complete!
-----------------------------------


In [4]:
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Wagar,SD,16.1525,36.2032,100.20,16,9,22.46,clear sky
1,Norman Wells,CA,65.2820,-126.8329,42.84,52,40,14.97,scattered clouds
2,Nago,JP,26.5881,127.9761,66.45,80,100,9.93,light rain
3,Kiryat Gat,IL,31.6100,34.7642,68.40,100,2,4.03,clear sky
4,Hermanus,ZA,-34.4187,19.2345,62.80,84,0,10.74,clear sky
5,Bluff,NZ,-46.6000,168.3333,44.80,91,20,8.75,few clouds
6,East London,ZA,-33.0153,27.9116,63.54,72,0,9.22,clear sky
7,Kapaa,US,22.0752,-159.3190,73.11,86,40,21.85,scattered clouds
8,Port Blair,IN,11.6667,92.7500,80.64,94,20,2.30,mist
9,Kamenka,RU,51.3223,42.7678,42.89,50,51,5.68,broken clouds


In [5]:
#export the city_data
city_data_df.to_csv('weatherPy_database.csv', index_label="City_ID")